In [ ]:
source("readTables.R")
source("progGraSpeech.R")

######################################################
# read Sight translation data ear-voice (interpretation)
ReadData("IMBi18/Tables/P05_I1")

# plot ear-voice garph
ProgGraph(stime = 1)

# read Sight translation data eye-voice (sight translation)
ReadData("IMBst18/Tables/P05_Ist1")

# plot eye-voice garph
ProgGraph()

### add voice data to eye-voice garph

# read interpretation (ear) data from ST file
ear <- read.table("IMBi18/Tables/P05_I1.st", "\t", header = TRUE)

# parameters: text(X, Y, char, color, ...)
text(ear$STime, ear$Id, "O", col="orange")


##########################
# zoom into the graph
ProgGraph(X1=1000, X2=30000)

# text(X, Y, char, color, ...)
text(ear$STime, ear$Id, "O", col="orange")


###############################################################
# compute ear-voice span, eye-voice span, ear-eye span
###############################################################
###############################################################
# - read the ear-voice tables (IMBi)
# - read the eye-voice tables (IMBst)
# - merge ear data (STime and SDur) into ear-voice tables

# read all session ST files 

EarST <- readCRITTables(c("IMBi18/Tables/"), ".st$", path=".")
EyeST <- readCRITTables(c("IMBst18/Tables/"), ".st$", path=".")

# check whether dimensions are identical
dim(EyeST)
dim(EarST)


#############################################
# merge ear-eye-voice data into one dataframe
# - create unique SessionTextId
# - sort tables according to SessionTextId (otherwise might be sorted differenty)
# - add voice data to Eye-Voice dataframe

# add unique ID for every word/text/translation
EarST$SessionTextId <- as.factor(paste(EarST$Session, EarST$Text, EarST$Id, sep = "_"))
EyeST$SessionTextId <- as.factor(paste(EyeST$Session, EarST$Text, EyeST$Id, sep = "_"))

# merge ST production times (ear: Stime and Sdur) into Eye-Voice dataframe
# - sort the dataframe 
EyeST <- EyeST[order(EyeST$SessionTextId), ]

# add the identically sorted columns
EyeST$STime <- EarST[order(EarST$SessionTextId), ]$STime
EyeST$SDur  <- EarST[order(EarST$SessionTextId), ]$Sdur

dim(EyeST)

# take out words with no translation (TGroup empty)
EyeST1 <- EyeST[(EyeST$TGroup != '---'), ]

# take out words with no production time 
EyeST1 <- EyeST[(EyeST$Time1 > 0), ]

# take out words with no fixation (FFTime == 0)
EyeST1 <- EyeST1[(EyeST1$FFTime > 0), ]

dim(EyeST1)

# Eye-Ear span: earTime - eyeTime
EyeST1$EyeEar <- EyeST1$STime - EyeST1$FFTime 

# Eye-Voice span: voiceTime - eyeTime 
EyeST1$EyeVoice <- EyeST1$Time1 - EyeST1$FFTime

# Ear-Voice span: voiceTime - earTime
EyeST1$EarVoice <- EyeST1$Time1 - EyeST1$STime 

# inspect distributions 
hist(EyeST1$EyeVoice, breaks = 100)
hist(EyeST1$EyeEar, breaks = 100)
hist(EyeST1$EarVoice, breaks = 100)

# relation between ears and eyes
plot(EyeST1$EarVoice, EyeST1$EyeVoice)

######################################################
# effect of HTra on various eye-ear-voice distances
# similar to the ST19 session

library(effects)
EyeST1$Task <- as.factor(EyeST1$Task)

LM0 <- lm(EyeVoice ~ HTra , data = EyeST1)
summary(LM0)
plot(allEffects(LM0), multiline=T)

LM1 <- lm(EarVoice ~ HTra , data = EyeST1)
summary(LM1)
plot(allEffects(LM1), multiline=T)

LM2 <- lm(EyeEar ~ HTra , data = EyeST1)
summary(LM2)
plot(allEffects(LM2), multiline=T)

######################################################
# compare this with the re-aligned IMB bolt set 
# what are the differences
